<a href="https://colab.research.google.com/github/claudio1975/SDS2020/blob/master/notebooks/laboratory/6a_Lab_An_Experimental_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **An Experimental Exploratory Data Analysis for a Classification Task step 6**

### ***From Visualization to Statistical Analysis***

### ***From Feature Engineering to Feature Selection***

### ***From the Best Model Selection to Interpretability***



To start the exploration set up the environment with libraries, upload the data set (it's stored in a github repository) and split it into target variable and features variables. No more set up is required using Google Colab. Look at the guidelines: https://colab.research.google.com/notebooks/welcome.ipynb

#### **Contents**

The goal of this challenge, launched by CrowdAnalytix, is to develop a model to predict whether a mortgage will be funded or not based on certain factors in a customer’s application data. 
The evaluation metric used is the F1 score.
The data set is made up by 45.642 observations with predictor variables (21 features) and the target variable. It's a classification task with the goal to predict the 'Result' target variable for every row (Funded, Not Funded). Look at the competition: https://www.crowdanalytix.com/contests/propensity-to-fund-mortgages


### **Exploratory Data Analysis (EDA) Pipeline**

![](http://www.theleader.info/wp-content/uploads/2017/07/Mortgage-rates.jpg)

# Prepare Workspace

#####- Upload libraries

In [ ]:
# Upload libraries

# to handle data set
import pandas as pd
import numpy as np

# to plot
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix

# statistics
import statistics
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import logit
from scipy.stats import chi2_contingency
from scipy.stats import kurtosis 
from scipy.stats import skew
from statistics import stdev 

# to split data set 
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build models
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# to evaluate models
from sklearn.metrics import f1_score

# to handle imbalanced data set
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# feature engineering
!pip install feature-engine
import feature_engine
from sklearn.preprocessing import KBinsDiscretizer

# feature importance
from sklearn.tree import DecisionTreeClassifier
!pip install eli5 
import eli5
from eli5.sklearn import PermutationImportance
!pip install shap
import shap
import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings('ignore')

#####- Upload data set

In [ ]:
# Upload dataset
url ='https://raw.githubusercontent.com/claudio1975/SDS2020/master/data/CAX_train_small.csv'
df = pd.read_csv(url)

#####- Split data set

In [ ]:
# Split data set between target and features
X_full = df
y = X_full.RESULT
X_full = X_full.drop(['RESULT'], axis=1)


# Summarize Data

In [ ]:
# Look at dimension of data set and types of each attribute
df.info()

In [ ]:
# Summarize attribute distributions of the data frame
df.describe(include='all')

In [ ]:
# Take a peek at the first rows of the data
df.head(10)

Explanatory variables are grouped into categorical variables and numerical variables and for each one let's do a graphical and non-graphical analysis, but before this split let's run some data preparation activities.

# Formatting Features

If necessary, it's a good practice to format data, after have taken a peek of it. Missing values on numeric features are marked by "-1", meanwhile for categorical features they are marked with "Unknown"; let's imput these values with "NA".  

In [ ]:
# Replaced both '-1' and 'Unknown' values with NA's
X_full[X_full== -1] = np.nan
X_full[X_full=="Unknown"] = np.nan

In [ ]:
# Format data into float and object types and split mixed variables
X_full['PROPERTY VALUE'] = X_full['PROPERTY VALUE'].astype(float)
X_full['MORTGAGE PAYMENT'] = X_full['MORTGAGE PAYMENT'].astype(float)
X_full['AMORTIZATION'] = X_full['AMORTIZATION'].astype(float)
X_full['TERM'] = X_full['TERM'].astype(float)
X_full['INCOME'] = X_full['INCOME'].astype(float)
X_full['INCOME TYPE'] = X_full['INCOME TYPE'].astype(object)
X_full['CREDIT SCORE'] = X_full['CREDIT SCORE'].astype(float)
X_full['FSA_num'] = X_full['FSA'].str.extract('(\d+)') # extract numerical part
X_full['FSA_let'] = X_full['FSA'].str[0] # extract the first letter

In [ ]:
# Rename some features for a practical use
X_full = X_full.rename(columns={"MORTGAGE PURPOSE":"MORTGAGE_PURPOSE","PAYMENT FREQUENCY":"PAYMENT_FREQUENCY","PROPERTY TYPE":"PROPERTY_TYPE","AGE RANGE":"AGE_RANGE","PROPERTY VALUE": "PROPERTY_VALUE",
                                "MORTGAGE PAYMENT": "MORTGAGE_PAYMENT", "MORTGAGE AMOUNT":"MORTGAGE_AMOUNT","INCOME TYPE":"INCOME_TYPE","CREDIT SCORE":"CREDIT_SCORE"})

# Handling missing values

There are two categorical features with missing values lower than 40%. The approach followed: filled up missing values with the mode of each variable. With large percentage of missing values (>=15%) it's suggested to add a "missing indicator", a boolean variable with 1/true (missing value) or 0/false (actual value).  "[Pawel Grabinski](https://www.kdnuggets.com/2018/12/feature-engineering-explained.html)"

In [ ]:
# Check missing values both to numeric features and categorical features
X_full.isnull().sum()/X_full.shape[0]*100

In [ ]:
# Input missing values with median or mode depending of features class
X_full['GENDER'].fillna(X_full['GENDER'].mode()[0], inplace=True)
X_full['GENDER'] = X_full['GENDER'].astype(object)
X_full['INCOME_TYPE'].fillna(X_full['INCOME_TYPE'].mode()[0], inplace=True)
X_full['INCOME_TYPE'] = X_full['INCOME_TYPE'].astype(object)

In [ ]:
# final check
X_full.isnull().sum()/X_full.shape[0]*100

# Target Variable Analysis

The target variable is grouped into two classes: "Funded" and "Not Funded". Looking at the barplot, it's quite imbalanced.

In [ ]:
# Summarize the class distribution 
count = pd.crosstab(index = y, columns="count")
percentage = pd.crosstab(index = y, columns="frequency")/pd.crosstab(index = y, columns="frequency").sum()
pd.concat([count, percentage], axis=1)

In [ ]:

# Plot the target variable
ax = sns.countplot(x=y, data=X_full, order=["FUNDED", "NOT FUNDED"]).set_title("Target Variable Distribution")

# Categorical Features Analysis


#####- Analysis for categorical features (barplot, univariate analysis, bivariate analysis)

Let's group all categorical features into a new subset: let's run a graphical analysis using barplots and count the frequency for each class. For a bivariate analysis it's been used a Chi-Square Test to evaluate the relationship between the target variable and each categorical feature. 

In [ ]:
# let's have a look at how many labels for categorical features
for col in X_full.columns:
  if X_full[col].dtype =="object":
    print(col, ': ', len(X_full[col].unique()), ' labels')

Unique_ID :  4564  labels
MORTGAGE_PURPOSE :  2  labels
PAYMENT_FREQUENCY :  3  labels
PROPERTY_TYPE :  8  labels
FSA :  753  labels
AGE_RANGE :  11  labels
GENDER :  2  labels
INCOME_TYPE :  16  labels
NAICS CODE :  19  labels
FSA_num :  10  labels
FSA_let :  16  labels


In [ ]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_full.columns if
                    X_full[cname].nunique() <= 15 and 
                    X_full[cname].dtype == "object"]

In [ ]:
# Subset with categorical features
cat = X_full[categorical_cols]
cat.columns


In [ ]:
cat.info()

In [ ]:
# Univariate analysis with frequency and barplots
sns.set( rc = {'figure.figsize': (5, 5)})
fcat = ['MORTGAGE_PURPOSE','PAYMENT_FREQUENCY','PROPERTY_TYPE','AGE_RANGE','GENDER','FSA_num']

for col in fcat:
    count = pd.crosstab(index = cat[col], columns="count")
    percentage = pd.crosstab(index = cat[col], columns="frequency")/pd.crosstab(index = cat[col], columns="frequency").sum()
    tab = pd.concat([count, percentage], axis=1)
    plt.figure()
    sns.countplot(x=cat[col], data=cat, palette="Set1")
    plt.xticks(rotation=45)
    print(tab)
    plt.show()

In [ ]:
# Bivariate analysis with barplots
sns.set( rc = {'figure.figsize': (5, 5)})
fcat = ['MORTGAGE_PURPOSE','PAYMENT_FREQUENCY','PROPERTY_TYPE','AGE_RANGE','GENDER','FSA_num']

for col in fcat:
    plt.figure()
    sns.countplot(x=cat[col], hue=y, data=cat, palette="Set1")
    plt.xticks(rotation=45)
    plt.show()

#####- Feature Selection

The Chi-Square Test is used as feature selection testing the independence between target variable and categorical features.

In [ ]:
# Joined target variable with categorical features in a new data frame for a Chi-Square Test
cat2 = pd.concat([y,cat], axis=1)
testColumns = ['MORTGAGE_PURPOSE','PAYMENT_FREQUENCY','PROPERTY_TYPE','AGE_RANGE','GENDER','FSA_num']

In [ ]:
alpha = 0.05
for var in testColumns:
    X = cat2[var].astype(str)
    Y = cat2['RESULT'].astype(str)
    dfObserved = pd.crosstab(Y,X)
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    if p <= alpha:
    	print('{0} Dependent (reject H0)'.format(var))
    else:
       print('{0} Independent (fail to reject H0)'.format(var))

In [ ]:
# Drop features not helpful by Feature Selection
cat = cat.drop(['GENDER','AGE_RANGE','FSA_num'], axis=1)
cat.shape

#####- Feature Engineering on categorical features: one-hot encoding into k-1 dummy variables

Let's transform categorical features into numerical variables with one-hot encoding methodology to afford a better understanding of variables by machine learning models.

In [ ]:
# One-hot encode the data (to shorten the code, I use pandas)
HOcat = pd.concat([pd.get_dummies(cat.MORTGAGE_PURPOSE, drop_first=True), pd.get_dummies(cat.PAYMENT_FREQUENCY, drop_first=True), pd.get_dummies(cat.PROPERTY_TYPE, drop_first=True)], axis=1)

HOcat = HOcat.astype(int)


In [ ]:
# Look at the new subset
HOcat.shape

In [ ]:
HOcat.head()

# Numerical Features Analysis

#####- Analysis for numerical features (distribution, univariate analysis, bivariate analysis)

In [ ]:
# Select numerical columns
numerical_cols = [cname for cname in X_full.columns if 
                X_full[cname].dtype in ['float64']]

In [ ]:
# Subset with numerical features
num = X_full[numerical_cols]
num.columns

Index(['PROPERTY_VALUE', 'MORTGAGE_PAYMENT', 'GDS', 'LTV', 'TDS',
       'AMORTIZATION', 'MORTGAGE_AMOUNT', 'RATE', 'TERM', 'INCOME',
       'CREDIT_SCORE'],
      dtype='object')

In [ ]:
num.info()

In [ ]:
# Univariate analysis with density plots and histograms
sns.set( rc = {'figure.figsize': (5, 5)})
fnum = ['PROPERTY_VALUE', 'MORTGAGE_PAYMENT','GDS', 'LTV', 'TDS', 'AMORTIZATION','MORTGAGE_AMOUNT', 'RATE', 'TERM', 'INCOME', 'CREDIT_SCORE']

for col in fnum:
    plt.figure()
    x=num[col]
    sns.distplot(x, bins=10, color='m')
    plt.xticks(rotation=45)
    plt.show()    

In [ ]:
# Univariate analysis with box-plots
for col in fnum:
    plt.figure()
    x=num[col]
    sns.boxplot(x,palette="Set1",linewidth=1)
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Univariate analysis looking at Standard Deviation, Skewness and Kurtosis
for col in fnum:
  print(col,'\nStandard Deviation :', stdev(num[col]), 
        '\nSkewness :', skew(num[col]), 
        '\nKurtosis :', kurtosis(num[col]))

In [ ]:
# Bivariate analysis with box-plots
for col in fnum:
    plt.figure()
    sns.violinplot(y=col, x=y, data=num, linewidth=2, palette="Set2")
    plt.show()

####- Feature Selection

The Anova Test on the Logistic Regression is used as feature selection testing the importance of the predictor variables.

In [ ]:
# Merging numerical covariates with dependent variable
num2 = pd.concat([y,num], axis=1)
num2['RESULT'] = np.where(num2['RESULT']=='FUNDED',1,0)

In [ ]:
# Anova Test on Logistic Regression
formula = ('RESULT ~ PROPERTY_VALUE+MORTGAGE_PAYMENT+GDS+LTV+TDS+AMORTIZATION+MORTGAGE_AMOUNT+RATE+TERM+INCOME+CREDIT_SCORE')
results = logit(formula=formula, data=num2).fit()
results.summary()

In [ ]:
# Drop features not helpful by Feature Selection
num = num.drop(['PROPERTY_VALUE','GDS','TDS','AMORTIZATION','MORTGAGE_AMOUNT','TERM','CREDIT_SCORE'], axis=1)
num.shape

####- Handling Outliers

An outlier is an observation that is unlike the other observations. They are extreme values that fall far away of the other observations. There are several ways to handle outliers, the approach followed is to cap them within two boundaries based on 1.5 times the Inter Quartile Range (difference between 75th and 25th quartiles).

In [ ]:
def outliers_plot(dataframe, feature):
    plt.figure(figsize=(15, 5))

    # histogram
    plt.subplot(1, 3, 1)
    sns.distplot(dataframe[feature], bins=30, color='g')
    plt.title('Histogram')
    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(dataframe[feature], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')
    # boxplot
    plt.subplot(1, 3, 3)
    x=dataframe[feature]
    sns.boxplot(x,linewidth=1.5, color='g')
    plt.title('Boxplot')

    plt.show()

In [ ]:
# MORTGAGE_PAYMENT
# before
outliers_plot(num, 'MORTGAGE_PAYMENT')

In [ ]:

# correction
i = 'MORTGAGE_PAYMENT'
q75, q25 = np.percentile(num[i].dropna(), [75 ,25])
iqr = q75 - q25 
min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5) 
num[i].loc[num[i] < min] = min
num[i].loc[num[i] > max] = max

In [ ]:
# MORTGAGE_PAYMENT
# after
outliers_plot(num, 'MORTGAGE_PAYMENT')

In [ ]:
# LTV
# before
outliers_plot(num, 'LTV')

In [ ]:
# correction
i = 'LTV'
q75, q25 = np.percentile(num[i].dropna(), [75 ,25])
iqr = q75 - q25 
min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5) 
num[i].loc[num[i] < min] = min
num[i].loc[num[i] > max] = max

In [ ]:
# after
outliers_plot(num, 'LTV')

In [ ]:
# RATE
# before
outliers_plot(num, 'RATE')

In [ ]:
# correction
i = 'RATE'
q75, q25 = np.percentile(num[i].dropna(), [75 ,25])
iqr = q75 - q25 
min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5) 
num[i].loc[num[i] < min] = min
num[i].loc[num[i] > max] = max

In [ ]:
# after
outliers_plot(num, 'RATE')

In [ ]:
# INCOME
# before
outliers_plot(num, 'INCOME')

In [ ]:
# correction
i = 'INCOME'
q75, q25 = np.percentile(num[i].dropna(), [75 ,25])
iqr = q75 - q25 
min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5) 
num[i].loc[num[i] < min] = min
num[i].loc[num[i] > max] = max

In [ ]:
# after
outliers_plot(num, 'INCOME')

####- Feature engineering on numerical variables

Feature engineering is a fundamental step in the data science process, because with right features the job of modeling is much easier and predictive outcome will perform better.

In [ ]:
# Build a new feature 
num['X1'] = (num['RATE']/100)*(num['LTV']/100)

In [ ]:
# before
sns.distplot(num['INCOME'], bins=10)
plt.show()

In [ ]:
# Transform a feature
num['INCOME'] = np.log(num['INCOME'])

In [ ]:
# after
sns.distplot(num['INCOME'], bins=10)
plt.show()

In [ ]:
# before
sns.distplot(num['RATE'], bins=10)
plt.show()

In [ ]:
# Discretization of "RATE" variable with K-means clustering
disc = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')

In [ ]:
disc.fit(num[['RATE']])

In [ ]:
num_tf_k = disc.transform(num[['RATE']])
num_tf_k = pd.DataFrame(num_tf_k, columns = ['RATE'])
num_tf_k['RATE'] = num_tf_k['RATE'].astype(int)

In [ ]:
# after
sns.countplot(num_tf_k['RATE'])
plt.show()

In [ ]:
# One-hot encode the 'RATE' feature (to shorten the code, I use pandas)
HOnum = pd.concat([num[['MORTGAGE_PAYMENT','LTV','X1','INCOME']], 
                   pd.get_dummies(num_tf_k.RATE, prefix='RATE', drop_first=True)], axis=1)

# Feature Selection on all data set

Another feature selection approach is to observe correlation between variables, let's apply it on all data set. There are some models such as linear regression where related features can deteriorate the performance (multicollinearity). Though some ensemble models are not sensitive at this topic, “Ensembles of tree-based models”, the approach followed is to remove them anyway because we don't know which model to use in advance.

In [ ]:
# Grasp all
X_all = pd.concat([HOcat, HOnum], axis=1, join='inner')

In [ ]:
# Correlation heatmap
corr_matrix = X_all.corr()
sns.set( rc = {'figure.figsize': (35, 35)})
plt.figure()
sns.heatmap(corr_matrix, square = True, annot=True, fmt='.2f')
plt.title('Correlation Heatmap on data set',size=30)
plt.yticks(fontsize="15")
plt.xticks(fontsize="15")
plt.show()

In [ ]:
# Select correlated features and removed it
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.75
to_drop = [column for column in upper.columns if any(upper[column].abs() > 0.75)]
to_drop

In [ ]:
# Drop features 
X_all = X_all.drop(X_all[to_drop], axis=1)

In [ ]:
# Look at the new data set
X_all.shape

In [ ]:
X_all.head()

In [ ]:
# Look for constant variables and drop them
for col in X_all.columns:
  print(col,'\nVariance :', np.var(X_all[col]))

# Split data set

To analyze the performance of a model is a good manner to split the data set into the training set and the test set. It's been decided to split it into three parts: training set, validation set and test set for a better understanding of models. The training set is a sample of data used to fit the model, meanwhile the validation set is a sample of data used to provide an unbiased evaluation of the model that fit on the training set and to tune the model hyperparameters (not in this explorative phase). The test set is a sample of data used to provide an unbiased evaluation of the model applied on data never seen before.

In [ ]:
# Break off validation and test set from training data
X_train, X_test, y_train, y_test = train_test_split(X_all, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Standardization

Since values of the features are not uniform and may be neagatively impact the skill of some models, the same models are evaluated with a standardized copy of the data set. It means, data are transformed such that each feature has a mean value of 0 and a standard deviation of 1. 

In [ ]:
# Standardization of data
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_valid_sc = sc.fit_transform(X_valid)
X_test_sc = sc.transform(X_test)

# Modeling Part

The traditional data exploration is extended looking at the behaviour of several baseline models and which features can be relevant for the prediction. This exploration is splitted in two parts: without handling the imbalanced target variable (scaled baseline models) and handling it (scaled baseline models).  

- Evaluation Metric and Confusion Matrix

The confusion matrix is a summary table representation of prediction results for a classification problem. The number of correct and incorrect predictions are summarized with count values and broken down by each class. The diagonal elements represent the number of points for which the predicted label is equal to the true label, while off-diagonal elements are those that are mislabeled by the classifier. Good predictions coming from the higher diagonal values of the confusion matrix. For this imbalanced classification task is not used Accuracy metric but more appropriately the F1 score metric that combines both precision and recall, it's an harmonic mean between them, it's indicates how precise is the classifier (precision) and how robust it is (recall). F1 score equal to 0.00 indicates a poor model, instead F1 score equal 1.00 indicates a perfect model.


#  Modeling Part I: without handling imbalanced data set

The analysis is based on six baseline models: Logistic Regression as the easiest model and as well as benchmark, then other five models: Bagging, Random Forest, AdaBoost, Gradient Boosting Machine and Neural Networks (MLP).

#####- Baseline Models

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LogisticRegression', LogisticRegression(random_state=0)))
models.append(('Bagging', BaggingClassifier(random_state=0)))
models.append(('RandomForest', RandomForestClassifier(random_state=0)))
models.append(('AdaBoost', AdaBoostClassifier(random_state=0)))
models.append(('GBM', GradientBoostingClassifier(random_state=0)))
models.append(('NN', MLPClassifier(random_state=0)))
results_tr = []
results_v = []
results_t = []
names = []
score = []
skf = StratifiedKFold(n_splits=5)
for (name, model) in models:
    param_grid = {}
    my_model = GridSearchCV(model,param_grid,cv=skf)
    my_model.fit(X_train_sc, y_train)
    predictions_tr = my_model.predict(X_train_sc) 
    predictions_v = my_model.predict(X_valid_sc)
    predictions_t = my_model.predict(X_test_sc)
    f1_train = f1_score(y_train, predictions_tr, average='macro') 
    f1_valid = f1_score(y_valid, predictions_v,average='macro') 
    f1_test = f1_score(y_test, predictions_t,average='macro') 
    results_tr.append(f1_train)
    results_v.append(f1_valid)
    results_t.append(f1_test) 
    names.append(name)
    f_dict = {
        'model': name,
        'f1_train': f1_train,
        'f1_valid': f1_valid,
        'f1_test': f1_test
    }
    score.append(f_dict)
    # Computing Confusion matrix for the above algorithms
    sns.set( rc = {'figure.figsize': (5, 5)})
    plt.figure()
    plot_confusion_matrix(my_model,X_test_sc, y_test,values_format= '.2f', cmap='Blues')
    plt.title(name)
    plt.show()   
score = pd.DataFrame(score, columns = ['model','f1_train', 'f1_valid', 'f1_test'])

In [ ]:
# Look at the F1 score for each model and for each data set
print(score)

In [ ]:
# Plot results for a graphical comparison
print("Spot Check Algorithms")
sns.set( rc = {'figure.figsize': (15, 5)})
plt.figure()
plt.subplot(1,3,1)  
sns.stripplot(x="model", y="f1_train",data=score,size=15)
plt.xticks(rotation=90)
plt.title('Train results')
axes = plt.gca()
axes.set_ylim([0,1.1])
plt.subplot(1,3,2)
sns.stripplot(x="model", y="f1_valid",data=score,size=15)
plt.xticks(rotation=90)
plt.title('Validation results')
axes = plt.gca()
axes.set_ylim([0,1.1])
plt.subplot(1,3,3)
sns.stripplot(x="model", y="f1_test",data=score,size=15)
plt.xticks(rotation=90)
plt.title('Test results')
axes = plt.gca()
axes.set_ylim([0,1.1])
plt.show()

In [ ]:
# Plot Precision - Recall curves for a graphical comparison
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
# train models
model_LR = LogisticRegression(random_state=0).fit(X_train_sc, y_train)
model_BAG = BaggingClassifier(DecisionTreeClassifier(random_state=0)).fit(X_train_sc, y_train)
model_RF = RandomForestClassifier(random_state=0).fit(X_train_sc, y_train)
model_AB = AdaBoostClassifier(DecisionTreeClassifier(random_state=0)).fit(X_train_sc, y_train)
model_GBM = GradientBoostingClassifier(random_state=0).fit(X_train_sc, y_train)
model_NN = MLPClassifier(random_state=0).fit(X_train_sc, y_train)
# precision-recall on train set
axes = plt.gca()
plot_precision_recall_curve(model_LR, X_train_sc, y_train, ax=axes)
plot_precision_recall_curve(model_BAG, X_train_sc, y_train, ax=axes)
plot_precision_recall_curve(model_RF, X_train_sc, y_train, ax=axes)
plot_precision_recall_curve(model_AB, X_train_sc, y_train, ax=axes)
plot_precision_recall_curve(model_GBM, X_train_sc, y_train, ax=axes)
plot_precision_recall_curve(model_NN, X_train_sc, y_train, ax=axes)
plt.legend(loc='lower left')
plt.show()


In [ ]:
# precision-recall on validation set
axes = plt.gca()
plot_precision_recall_curve(model_LR, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_BAG, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_RF, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_AB, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_GBM, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_NN, X_valid_sc, y_valid, ax=axes)
plt.legend(loc='upper right')
plt.show()


In [ ]:
# precision-recall on test set
axes = plt.gca()
plot_precision_recall_curve(model_LR, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_BAG, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_RF, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_AB, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_GBM, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_NN, X_test_sc, y_test, ax=axes)
plt.legend(loc='upper right')
plt.show()

# Oversampling

To overcome the imbalanced classification the simplest strategy is to apply a random resampling strategy. It's been used a random oversampling that consists on create a new transformed data set with randomly duplicate examples in the minority class.

In [ ]:
# Oversampling
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

[('FUNDED', 2268), ('NOT FUNDED', 2268)]


# Standardization

In [ ]:
# Standardization of data
sc = StandardScaler()
X_train_sc_re = sc.fit_transform(X_resampled)


#  Modeling Part II: handling imbalanced data set

#####- Baseline Models

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LogisticRegression', LogisticRegression(random_state=0)))
models.append(('Bagging', BaggingClassifier(random_state=0)))
models.append(('RandomForest', RandomForestClassifier(random_state=0)))
models.append(('AdaBoost', AdaBoostClassifier(random_state=0)))
models.append(('GBM', GradientBoostingClassifier(random_state=0)))
models.append(('NN', MLPClassifier(random_state=0)))
results_tr = []
results_v = []
results_t = []
names = []
score = []
skf = StratifiedKFold(n_splits=5)
for (name, model) in models:
    param_grid = {}
    my_model = GridSearchCV(model,param_grid,cv=skf)
    my_model.fit(X_train_sc_re, y_resampled)
    predictions_tr = my_model.predict(X_train_sc_re) 
    predictions_v = my_model.predict(X_valid_sc)
    predictions_t = my_model.predict(X_test_sc)
    f1_train = f1_score(y_resampled, predictions_tr, average='macro') 
    f1_valid = f1_score(y_valid, predictions_v,average='macro') 
    f1_test = f1_score(y_test, predictions_t,average='macro') 
    results_tr.append(f1_train)
    results_v.append(f1_valid)
    results_t.append(f1_test)
    
    names.append(name)
    f_dict = {
        'model': name,
        'f1_train': f1_train,
        'f1_valid': f1_valid,
        'f1_test': f1_test
    }
    score.append(f_dict)
    # Computing Confusion matrix for the above algorithms
    sns.set( rc = {'figure.figsize': (5, 5)})
    plt.figure()
    plot_confusion_matrix(my_model,X_test_sc, y_test,values_format= '.2f', cmap='Greys')
    plt.title(name)
    plt.show()   
score = pd.DataFrame(score, columns = ['model','f1_train', 'f1_valid', 'f1_test'])

In [ ]:
# Look at the F1 score for each model and for each data set
print(score)

In [ ]:
# Plot results for a graphical comparison
print("Spot Check Algorithms")
sns.set( rc = {'figure.figsize': (15, 5)})
plt.figure()
plt.subplot(1,3,1)  
sns.stripplot(x="model", y="f1_train",data=score,size=15)
plt.xticks(rotation=90)
plt.title('Train results')
axes = plt.gca()
axes.set_ylim([0,1.1])
plt.subplot(1,3,2)
sns.stripplot(x="model", y="f1_valid",data=score,size=15)
plt.xticks(rotation=90)
plt.title('Validation results')
axes = plt.gca()
axes.set_ylim([0,1.1])
plt.subplot(1,3,3)
sns.stripplot(x="model", y="f1_test",data=score,size=15)
plt.xticks(rotation=90)
plt.title('Test results')
axes = plt.gca()
axes.set_ylim([0,1.1])
plt.show()

In [ ]:
# Plot Precision - Recall curves for a graphical comparison
# train models
model_LR_re = LogisticRegression(random_state=0).fit(X_train_sc_re, y_resampled)
model_BAG_re = BaggingClassifier(DecisionTreeClassifier(random_state=0)).fit(X_train_sc_re, y_resampled)
model_RF_re = RandomForestClassifier(random_state=0).fit(X_train_sc_re, y_resampled)
model_AB_re = AdaBoostClassifier(DecisionTreeClassifier(random_state=0)).fit(X_train_sc_re, y_resampled)
model_GBM_re = GradientBoostingClassifier(random_state=0).fit(X_train_sc_re, y_resampled)
model_NN_re = MLPClassifier(random_state=0).fit(X_train_sc_re, y_resampled)
# precision-recall on train set
axes = plt.gca()
plot_precision_recall_curve(model_LR_re, X_train_sc_re, y_resampled, ax=axes)
plot_precision_recall_curve(model_BAG_re, X_train_sc_re, y_resampled, ax=axes)
plot_precision_recall_curve(model_RF_re, X_train_sc_re, y_resampled, ax=axes)
plot_precision_recall_curve(model_AB_re, X_train_sc_re, y_resampled, ax=axes)
plot_precision_recall_curve(model_GBM_re, X_train_sc_re, y_resampled, ax=axes)
plot_precision_recall_curve(model_NN_re, X_train_sc_re, y_resampled, ax=axes)
plt.legend(loc='lower left')
plt.show()


In [ ]:
# precision-recall on validation set
axes = plt.gca()
plot_precision_recall_curve(model_LR_re, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_BAG_re, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_RF_re, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_AB_re, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_GBM_re, X_valid_sc, y_valid, ax=axes)
plot_precision_recall_curve(model_NN_re, X_valid_sc, y_valid, ax=axes)
plt.legend(loc='upper right')
plt.show()


In [ ]:
# precision-recall on test set
axes = plt.gca()
plot_precision_recall_curve(model_LR_re, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_BAG_re, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_RF_re, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_AB_re, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_GBM_re, X_test_sc, y_test, ax=axes)
plot_precision_recall_curve(model_NN_re, X_test_sc, y_test, ax=axes)
plt.legend(loc='upper right')
plt.show()

# Feature Importance Analysis

Feature importance answers to this question "What features have the biggest impact on predictions?" Are showed feature importance from models, by permutation or by SHAP values. Permutation is based on understand the magnitude of each feature shuffling it per time and looking how the prediction change. SHAP values exploits the game thoery to measure how much each feature in the model contributes, either positively or negatively, to a predicted score. 

### ***Logistic Regression***

In [ ]:
# Top relevant features Logistic Regression
#model_LR = LogisticRegression(random_state=0)
#model_LR.fit(X_train_sc_re, y_resampled)
feature_importances = model_LR_re.coef_[0]

(pd.Series(feature_importances, index=X_train.columns).nlargest(10).plot(kind='bar')).set_title("Logistic Regression")

#####- Feature Permutation

In [ ]:
# Permutation importance Logistic Regression
print('Logistic Regression Features Permutation Importance')
perm = PermutationImportance(model_LR_re, random_state=0).fit(X_train_sc_re, y_resampled)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

### ***Bagging***

#####- Feature Importance

In [ ]:
# Top relevant features Bagging
#model_BAG = BaggingClassifier(DecisionTreeClassifier(random_state=0))
#model_BAG.fit(X_train_sc_re, y_resampled)
feature_importances = np.mean([
    tree.feature_importances_ for tree in model_BAG_re.estimators_
], axis=0)

(pd.Series(feature_importances, index=X_train.columns).nlargest(10).plot(kind='bar')).set_title("Bagging")

#####- Feature Permutation

In [ ]:
# Permutation importance Bagging
print('Bagging Features Permutation Importance')
perm = PermutationImportance(model_BAG_re, random_state=0).fit(X_train_sc_re, y_resampled)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

### ***Random Forest***

#####- Feature Importance

In [ ]:
# Top relevant features RF
#model_RF = RandomForestClassifier()
#model_RF.fit(X_train_sc_re, y_resampled)
(pd.Series(model_RF_re.feature_importances_, index=X_train.columns).nlargest(10).plot(kind='bar')).set_title("RF Features Importance")

#####- Feature Permutation

In [ ]:
# Permutation importance RF
print('RF Features Permutation Importance')
perm = PermutationImportance(model_RF_re, random_state=0).fit(X_train_sc_re, y_resampled)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

### ***AdaBoost***

#####- Feature Importance

In [ ]:
# Top relevant features AdaBoost
#model_AB = AdaBoostClassifier(DecisionTreeClassifier(random_state=0))
#model_AB.fit(X_train_sc_re, y_resampled)
feature_importances = np.mean([
    tree.feature_importances_ for tree in model_AB_re.estimators_
], axis=0)

(pd.Series(feature_importances, index=X_train.columns).nlargest(10).plot(kind='bar')).set_title("AdaBoost")

#####- Feature Permutation

In [ ]:
# Permutation importance AdaBoost
print('AdaBoost Features Permutation Importance')
perm = PermutationImportance(model_AB_re, random_state=0).fit(X_train_sc_re, y_resampled)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

### ***GBM***

#####- Feature Importance

In [ ]:
# Top relevant features GBM
#model_GBM = GradientBoostingClassifier()
#model_GBM.fit(X_train_sc_re, y_resampled)
(pd.Series(model_GBM_re.feature_importances_, index=X_train.columns).nlargest(10).plot(kind='bar')).set_title("GBM Features Importance")

#####- Feature Permutation

In [ ]:
# Permutation importance GBM
print('GBM Features Permutation Importance')
perm = PermutationImportance(model_GBM_re, random_state=0).fit(X_train_sc_re, y_resampled)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

#####- SHAP Values Analysis

In [ ]:
# SHAP Values Interpretability GBM
print("GBM SHAP Values")
shap_values = shap.TreeExplainer(model_GBM_re).shap_values(X_train_sc_re)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, features= X_train_sc_re, feature_names=X_train.columns)

### ***NN***

#####- Feature Permutation

In [ ]:
# Permutation importance NN
print('NN Features Permutation Importance')
#model_NN = MLPClassifier(random_state=0)
#model_NN.fit(X_train_sc_re, y_resampled)
perm = PermutationImportance(model_NN_re, random_state=0).fit(X_train_sc_re, y_resampled)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())